In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error, r2_score
from scipy.stats import skew, kurtosis
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.svm import SVR
from sklearn.preprocessing import LabelEncoder

from keras.models import Sequential
from keras.layers import Dense
import tensorflow as tf
from tensorflow import keras
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, GridSearchCV

from mpl_toolkits.basemap import Basemap
from collections import OrderedDict
import datetime


In [2]:
data_clean= pd.read_csv("data_clean .csv", sep=',')
data_clean

,Month,Day,DayOfWeek,UniqueCarrier,TailNum,DepDelay,Origin,Dest,TaxiOut,Date,CRSDepTime
0,1,28,5,HP,N808AW,0.0,ATL,PHX,11,2000-01-28,16:47:00
1,1,29,6,HP,N653AW,1.0,ATL,PHX,47,2000-01-29,16:47:00
2,1,31,1,HP,N806AW,-2.0,ATL,PHX,14,2000-01-31,16:47:00
3,1,1,6,HP,N158AW,-4.0,ATL,PHX,8,2000-01-01,08:46:00
4,1,2,7,HP,N656AW,3.0,ATL,PHX,24,2000-01-02,08:46:00
...,...,...,...,...,...,...,...,...,...,...,...
1003329,3,15,3,TW,N979Z,5.0,STL,LNK,14,2000-03-15,11:27:00
1003330,3,16,4,TW,N983Z,5.0,STL,LNK,20,2000-03-16,11:27:00
1003331,3,17,5,TW,N996Z,-2.0,STL,LNK,18,2000-03-17,11:27:00
1003332,3,18,6,TW,N982PS,-1.0,STL,LNK,11,2000-03-18,11:27:00


## <font color='#922B21'> Part 4: Feature Selection et Feature engineering </font>

Après notre étude dans la partie précedente, nous pouvons conclure qu'on a au final 4 variables importantes qui contribuent au retrat de départ qui sont la compagnie aèrienne, l'origine et destinationdu vol,et enfin l'heure, la date prévue pour le départ et de l'avion lui meme.

On va ignoré le variable TaxiIn puisque sa correlation avec le variable à prédire est tès faible.

Dans cette partie on va préparer ces variables pour la partie finale.

## <font color='  #d98880'>4.1.Encodage des variables `UniqueCarrier`

In [3]:
# Initialiser le LabelEncoder
le = LabelEncoder()

# Ajuster le LabelEncoder sur les données de 'UniqueCarrier'
le.fit(data_clean['UniqueCarrier'])

# Transformer la colonne 'UniqueCarrier'
data_clean['UniqueCarrier_encoded'] = le.transform(data_clean['UniqueCarrier'])


# Supprimer la colonne originale si nécessaire
data_clean.drop(columns=['UniqueCarrier'], inplace=True)

data_clean

,Month,Day,DayOfWeek,TailNum,DepDelay,Origin,Dest,TaxiOut,Date,CRSDepTime,UniqueCarrier_encoded
0,1,28,5,N808AW,0.0,ATL,PHX,11,2000-01-28,16:47:00,4
1,1,29,6,N653AW,1.0,ATL,PHX,47,2000-01-29,16:47:00,4
2,1,31,1,N806AW,-2.0,ATL,PHX,14,2000-01-31,16:47:00,4
3,1,1,6,N158AW,-4.0,ATL,PHX,8,2000-01-01,08:46:00,4
4,1,2,7,N656AW,3.0,ATL,PHX,24,2000-01-02,08:46:00,4
...,...,...,...,...,...,...,...,...,...,...,...
1003329,3,15,3,N979Z,5.0,STL,LNK,14,2000-03-15,11:27:00,6
1003330,3,16,4,N983Z,5.0,STL,LNK,20,2000-03-16,11:27:00,6
1003331,3,17,5,N996Z,-2.0,STL,LNK,18,2000-03-17,11:27:00,6
1003332,3,18,6,N982PS,-1.0,STL,LNK,11,2000-03-18,11:27:00,6


## <font color='  #d98880'>4.2.Encodage des variables `Origin` et `Dest`

In [4]:
# Initialiser le LabelEncoder
le = LabelEncoder()

# Concaténer les colonnes Origin et Dest pour avoir un encodage cohérent
all_airports = pd.concat([data_clean['Origin'], data_clean['Dest']])

# Ajuster le LabelEncoder sur les données concaténées
le.fit(all_airports)

# Transformer les colonnes Origin et Dest
data_clean['Origin_encoded'] = le.transform(data_clean['Origin'])
data_clean['Dest_encoded'] = le.transform(data_clean['Dest'])

# Supprimer les colonnes originales si nécessaire
data_clean.drop(columns=['Origin', 'Dest'], inplace=True)

## <font color='  #d98880'>4.3.Encodage du variable `TailNum`

In [5]:
# Créer une instance de LabelEncoder
label_encoder_tailnum = LabelEncoder()

# Encoder la colonne TailNum
data_clean['TailNum_encoded'] = label_encoder_tailnum.fit_transform(data_clean['TailNum'])


## <font color='  #d98880'>4.4.Extraction des variables à partir du temps de retard prévue`CRSDepTime` 

In [6]:
# Convertir la variable CRSDepTime en format datetime

data_clean['CRSDepTime'] = pd.to_datetime(data_clean['CRSDepTime'])

# Extraire les heures et les minutes
data_clean['Heure_Depart_Prevue'] = data_clean['CRSDepTime'].dt.hour
data_clean['Minute_Depart_Prevue'] = data_clean['CRSDepTime'].dt.minute



C:\Users\Lenovo\AppData\Local\Temp\ipykernel_13720\1267367563.py:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  data_clean['CRSDepTime'] = pd.to_datetime(data_clean['CRSDepTime'])


In [7]:
data_clean

,Month,Day,DayOfWeek,TailNum,DepDelay,TaxiOut,Date,CRSDepTime,UniqueCarrier_encoded,Origin_encoded,Dest_encoded,TailNum_encoded,Heure_Depart_Prevue,Minute_Depart_Prevue
0,1,28,5,N808AW,0.0,11,2000-01-28,2024-06-08 16:47:00,4,8,149,2985,16,47
1,1,29,6,N653AW,1.0,47,2000-01-29,2024-06-08 16:47:00,4,8,149,2381,16,47
2,1,31,1,N806AW,-2.0,14,2000-01-31,2024-06-08 16:47:00,4,8,149,2977,16,47
3,1,1,6,N158AW,-4.0,8,2000-01-01,2024-06-08 08:46:00,4,8,149,266,8,46
4,1,2,7,N656AW,3.0,24,2000-01-02,2024-06-08 08:46:00,4,8,149,2396,8,46
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1003329,3,15,3,N979Z,5.0,14,2000-03-15,2024-06-08 11:27:00,6,184,110,3617,11,27
1003330,3,16,4,N983Z,5.0,20,2000-03-16,2024-06-08 11:27:00,6,184,110,3639,11,27
1003331,3,17,5,N996Z,-2.0,18,2000-03-17,2024-06-08 11:27:00,6,184,110,3681,11,27
1003332,3,18,6,N982PS,-1.0,11,2000-03-18,2024-06-08 11:27:00,6,184,110,3629,11,27


In [8]:
from IPython.display import FileLink
# Enregistrer le dataframe au format CSV
data_clean.to_csv("data_clean.csv", index=False)

# Afficher le lien de téléchargement
FileLink("data_clean.csv")


C:\Users\Lenovo\data_clean.csv

## <font color='#922B21'> Part 5: Modeling </font>

In [9]:
columns = ['Month', 'DayOfWeek', 'TailNum_encoded', 'DepDelay', 'UniqueCarrier_encoded', 'Origin_encoded', 'Dest_encoded', 'Heure_Depart_Prevue', 'Minute_Depart_Prevue']
data_clean_encoded = data_clean[columns]
data_clean_encoded

,Month,DayOfWeek,TailNum_encoded,DepDelay,UniqueCarrier_encoded,Origin_encoded,Dest_encoded,Heure_Depart_Prevue,Minute_Depart_Prevue
0,1,5,2985,0.0,4,8,149,16,47
1,1,6,2381,1.0,4,8,149,16,47
2,1,1,2977,-2.0,4,8,149,16,47
3,1,6,266,-4.0,4,8,149,8,46
4,1,7,2396,3.0,4,8,149,8,46
...,...,...,...,...,...,...,...,...,...
1003329,3,3,3617,5.0,6,184,110,11,27
1003330,3,4,3639,5.0,6,184,110,11,27
1003331,3,5,3681,-2.0,6,184,110,11,27
1003332,3,6,3629,-1.0,6,184,110,11,27


## <font color='  #d98880'>5.1.Approche machine learnig 

**Random Forest: Pourquoi?**

On a décidé d'utiliser la méthode des forêts aléatoires pour détecter les retards de vols dans un aéroport en raison de plusieurs avantages spécifiques qu'elle offre dans ce contexte. L'algorithme des forêts aléatoires est une extension de la méthode du bagging, car il utilise à la fois le bagging et la randomisation des caractéristiques pour créer une forêt non corrélée d'arbres de décision. La randomisation des caractéristiques, également connue sous le nom de "feature bagging" ou "random subspace method", génère un sous-ensemble aléatoire de caractéristiques, ce qui garantit une faible corrélation entre les arbres de décision. Cela représente une différence clé par rapport aux arbres de décision simples, qui considèrent toutes les divisions de caractéristiques possibles, tandis que les forêts aléatoires ne sélectionnent qu'un sous-ensemble de ces caractéristiques.

En pratique, cela signifie que les forêts aléatoires sont capables de capturer une plus grande diversité de patterns dans les données, ce qui est crucial pour prédire les retards de vols, car ces retards peuvent être influencés par une multitude de facteurs variés comme les conditions météorologiques, le trafic aérien, les horaires des compagnies aériennes, et bien d'autres. En générant des arbres de décision non corrélés, l'algorithme réduit le risque de surapprentissage (overfitting), où le modèle est trop ajusté aux données d'entraînement et ne généralise pas bien aux nouvelles données. De plus, en réduisant le biais et la variance, les prédictions deviennent plus précises et robustes.

**Fonctionnement du Random Forest** 

Pour expliquer brièvement le fonctionnement de l'algorithme des forêts aléatoires, voici les étapes principales : d'abord, plusieurs échantillons de données sont créés en utilisant le bagging (bootstrap aggregating). Ensuite, pour chaque échantillon, un arbre de décision est construit, mais à chaque division de l'arbre, seule une sous-ensemble aléatoire des caractéristiques est considéré. Une fois que tous les arbres de la forêt sont construits, la prédiction finale pour un nouveau point de données est obtenue en agrégant les prédictions de tous les arbres, souvent par un vote majoritaire pour les problèmes de classification ou par la moyenne pour les problèmes de régression.

Ainsi, l'utilisation des forêts aléatoires dans la détection des retards de vols permet d'obtenir des modèles robustes et fiables, capables de gérer la complexité et la variabilité des facteurs influençant les retards, tout en minimisant les risques d'erreurs de prédiction.

**séparation des features et target**

In [10]:
# Séparer les caractéristiques (features) et la variable cible (target)
X = data_clean_encoded.drop('DepDelay', axis=1)
y = data_clean_encoded['DepDelay']

**Division des données d'entrainement et de test**

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

**Initialisation du modèle**

* n_estimators : Le nombre d'arbres dans la forêt.
* criterion : La fonction pour mesurer la qualité d'un split (séparation).
* min_samples_split : Le nombre minimum d'échantillons nécessaires pour diviser un noeud.
* min_samples_leaf : Le nombre minimum d'échantillons nécessaires pour être une feuille.

In [15]:
random_forest_model = RandomForestRegressor(
    n_estimators=100, 
    max_depth=10,
    criterion='squared_error',
    min_samples_split=2,
    min_samples_leaf=1,
    random_state=42
)

**Entrainement du modèle et prédiction**

In [16]:
# Entraîner le modèle sur les données d'entraînement
random_forest_model.fit(X_train, y_train)

# Prédire sur les données de test
y_pred =random_forest_model.predict(X_test)

**Evaluation du modèle**

In [ ]:
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error: {mse:.2f}")
print(f"R^2 Score: {r2:.2f}")

**Learning Curve pour déterminer comment notre modèle évolue en fonction de la taille de l'nsemble d'entrainement**

In [ ]:
train_sizes = np.linspace(0.1, 1.0, 10)  # 10 tailles différentes
from sklearn.model_selection import learning_curve
# Calculer les scores d'apprentissage et de validation croisée pour différentes tailles d'ensemble d'entraînement
train_sizes, train_scores, validation_scores = learning_curve(
   random_forest_model, X_train, y_train, train_sizes=train_sizes, cv=5, scoring='neg_mean_squared_error')


In [ ]:
# Calculer la moyenne des scores pour chaque taille d'ensemble d'entraînement
train_scores_mean = -train_scores.mean(axis=1)
validation_scores_mean = -validation_scores.mean(axis=1)

# Tracer la courbe d'apprentissage
plt.figure(figsize=(10, 6))
plt.plot(train_sizes, train_scores_mean, label='Training error')
plt.plot(train_sizes, validation_scores_mean, label='Validation error')
plt.xlabel('Training set size')
plt.ylabel('Mean Squared Error')
plt.title('Learning Curve')
plt.legend()
plt.grid(True)
plt.show()

**K-Fold**

cross-validation) est une technique utilisée pour évaluer les performances d'un modèle en utilisant des sous-ensembles de données distincts pour l'entraînement et la validation, ce qui permet d'estimer la capacité du modèle à généraliser à de nouvelles données. L'une des méthodes les plus courantes de validation croisée est la validation croisée k-fold.

In [ ]:
from sklearn.model_selection import cross_val_score

num_folds = 5

# Utilisez la fonction cross_val_score pour obtenir les scores de validation croisée
cross_val_scores = cross_val_score(random_forest_model, X, y, cv=num_folds, scoring='neg_mean_squared_error')

# Les scores sont négatifs, donc prenez leur moyenne pour obtenir la MSE
mean_mse = -cross_val_scores.mean()

# Affichez le résultat
print(f'Mean Cross-Validation MSE: {mean_mse}')